Assignment 1: Web scrapping 

imports

In [25]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

make http request to get html document

In [26]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)

construct html parser

In [27]:
soup = BeautifulSoup(response.text, "html.parser")

parse/scrape html content into dataframe

In [4]:
import pandas as pd
df = pd.DataFrame(columns=["PostalCode","Borough","Neighborhood"])
df.set_index("PostalCode", inplace=True)
             
table1 = soup.find('table', class_="wikitable sortable")
trs = table1.findAll('tr')
for i in range(0,len(trs)-1):
    tr1 = trs[i]
    tds = tr1.findAll('td')
    if (len(tds)==3 and tds[1].text.strip()!="Not assigned"):
        postalCode=tds[0].text.strip()
        borough=tds[1].text.strip()
        neighborhood=tds[2].text.strip()
        if neighborhood=="Not assigned":
            neighborhood = borough
        else:
            neighborhood = neighborhood.replace("\n","")
        
        try:
            df.loc[postalCode]["Neighborhood"] = df.loc[postalCode]["Neighborhood"] + ", " + neighborhood 
        except: 
            df.loc[postalCode] = [borough, neighborhood] 

print(df.shape)        
df.head()

(103, 2)


,Borough,Neighborhood
PostalCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Queen's Park


Assignment 2: geocoder

add columns to dataframe for lat,lng location

In [5]:
df["Latitude"]=float('nan')
df["Longitude"]=float('nan')
df.dtypes

Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

geocoder does not work, so just load the lookup dataset

In [6]:
#Error using geocoder 
#!pip install geocoder
#import geocoder
#geocoder.google('M3A, Toronto, Ontario')
#error <[REQUEST_DENIED] Google - Geocode [empty]>
postal_df = pd.read_csv("http://cocl.us/Geospatial_data")
print(postal_df.shape)
postal_df.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


prepare for looking-up & setting lat,lng locations

In [7]:
df.reset_index(inplace=True)
postal_df.set_index("Postal Code", inplace=True)


lookup and set lat,lng

In [8]:

for i in range(0,df.shape[0]):
    postalCode = df.iloc[i].PostalCode
    try:
        [df.iloc[i,3], df.iloc[i,4]] = postal_df.loc[postalCode]
    except:
        print("{} lookup failed.", postalCode)


reverse what I did on index in preparation above

In [9]:
df.set_index('PostalCode', inplace=True)
postal_df.reset_index(inplace=True)

In [10]:
df.head()


,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [11]:
df.tail()

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999


Assignment 3: cluster the neighborhoods in Toronto

import required libs

In [12]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    altair-3.1.0               |           py36_0         724 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

foursquare credentials

In [13]:
{"tags": ["hide_input", ]}
CLIENT_ID = 'OQVCM4PDX12QZZZPZP2T5PWPUDNRMHLAGDASLUD1BZX4MJYX' # your Foursquare ID
CLIENT_SECRET = 'VOXCDZHSZLAT4U41UZNBQ5GUTK2TFRY4I5QTUXWJVGQTBBOD' # your Foursquare Secret
VERSION = '20180605'
LIMIT=100

find Toronto Lat,Lng

In [14]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
toronto_lat = location.latitude
toronto_lng = location.longitude
print(toronto_lat, toronto_lng)



43.653963 -79.387207


show zip center map toronto

In [15]:
map_Toronto = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=10)


In [16]:
# add neighborhood markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  

In [17]:
# show map
map_Toronto

function for exploring neighborhoods - per postal code.

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        
            # return only relevant information for each nearby venue
            for v in results:
                try:
                    venues_list.append([(
                    name, 
                    lat, 
                    lng, 
                    v['venue']['name'], 
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng'],  
                    v['venue']['categories'][0]['name'])])
                except:
                    print("except on {} v={} ".format(name, v ))
        except:
            print('except')
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [19]:
df.reset_index(inplace=True)  # needed to use df['Neighborhood'] in next cell

In [20]:
# run the function
neighborhood_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

examine explored results

In [28]:
print(neighborhood_venues.shape)
neighborhood_venues.head()


(2252, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
3,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


add category per neighborhood group

In [29]:
onehot = pd.get_dummies(neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
onehot['Neighborhood'] = neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot2 = onehot[fixed_columns]
onehot_grouped = onehot2.groupby('Neighborhood').mean().reset_index()

find top 10 venues each neighborhood

In [37]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(0, 3):
    columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
for ind in np.arange(3, num_top_venues):
    columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = onehot_grouped['Neighborhood']

for ind in np.arange(onehot_grouped.shape[0]):
    t1_sort=onehot_grouped.iloc[ind, :].iloc[1:]
    t1_sort.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 1:] = t1_sort.index.values[0: num_top_venues]

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()


(100, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
1,Agincourt,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
4,"Alderwood, Long Branch",Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant


Clustering neighbothoods

In [38]:
# set number of clusters
kclusters = 5

onehot_grouped_clustering = onehot_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(onehot_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

label clustered results for map visualization

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
onehot_merged = df.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

onehot_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0.0,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.0,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant


visualize it on map

In [60]:
# create map
map_clusters = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(onehot_merged['Latitude'], onehot_merged['Longitude'], onehot_merged['Neighborhood'], onehot_merged['Cluster Labels']):
    try:
        cluster_i = int(cluster)
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster_i), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster_i-1],
            fill=True,
            fill_color=rainbow[cluster_i-1],
            fill_opacity=0.7).add_to(map_clusters)
    except:
        print("bad cluster at poi {}: {}".format(poi, cluster))
map_clusters



bad cluster at poi Islington Avenue: nan
bad cluster at poi Newtonbrook, Willowdale: nan
bad cluster at poi Upper Rouge: nan
